<a href="https://colab.research.google.com/github/rahulsm27/ML/blob/main/Text_Generation_using_Fnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install datasets

In [5]:
import torch

In [6]:
from datasets import load_dataset
datasets = load_dataset('wikitext','wikitext-2-raw-v1')

In [7]:
datasets

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [8]:
import re
def preprocess_text(sentence):
  text = sentence['text'].lower() # lowering the sentence and storing in text vaiable
  text = re.sub('[^a-z?!.,]',' ',text) # removing other than characters and punctuations
  text = re.sub('\s\s+',' ',text) # removing double spaces
  sentence['text'] = text
  return sentence

In [9]:
datasets['train'] = datasets['train'].map(preprocess_text)
datasets['test'] = datasets['test'].map(preprocess_text)
datasets['validation'] = datasets['validation'].map(preprocess_text)

In [10]:
datasets['train'] = datasets['train'].filter(lambda x : len(x['text']) > 20)
datasets['test'] = datasets['test'].filter(lambda x : len(x['text']) > 20)
datasets['validation'] = datasets['validation'].filter(lambda x : len(x['text']) > 20)

In [11]:
datasets

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 2312
    })
    train: Dataset({
        features: ['text'],
        num_rows: 18794
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 1988
    })
})

In [13]:
 from torchtext.data.utils import get_tokenizer
 tokenizer = get_tokenizer("spacy")
 tokenizer('I have two asfdf')

OSError: ignored

In [ ]:

from collections import Counter



counter = Counter()
for lines in datasets['train']:
    counter.update(tokenizer(lines['text']))


In [ ]:
from torchtext.vocab import vocab
special_tokens = ['<pad>', '<bos>', '<eos>', '<unk>']
vocab = vocab(counter=counter, specials=special_tokens, unk_init=torch.Tensor.normal_)

In [32]:
!pip install update torch

In [1]:
torch.__version__

NameError: ignored

In [42]:
!pip install torch==2.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 669.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [3]:
!pip show torch


Name: torch
Version: 2.0.1
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, jinja2, networkx, nvidia-cublas-cu11, nvidia-cuda-cupti-cu11, nvidia-cuda-nvrtc-cu11, nvidia-cuda-runtime-cu11, nvidia-cudnn-cu11, nvidia-cufft-cu11, nvidia-curand-cu11, nvidia-cusolver-cu11, nvidia-cusparse-cu11, nvidia-nccl-cu11, nvidia-nvtx-cu11, sympy, triton, typing-extensions
Required-by: fastai, torchaudio, torchdata, torchtext, torchvision, triton


In [ ]:
!pip